In [8]:
import re
import pandas as pd
import numpy as np
import torch
import os
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
from tqdm.notebook import tqdm
from collections import Counter
# from wordcloud import WordCloud
# from konlpy.tag import Okt $23
import matplotlib.font_manager as fm
import urllib.request
from urllib.parse import urlparse,parse_qs
import requests
from bs4 import BeautifulSoup
import json
import time
import datetime
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
import unicodedata
import networkx as nx
from sklearn.manifold import MDS

<h1 style="margin:5px;">파이썬 데이터 분석- "탄핵" 키워드의 네이버 뉴스 댓글을 통한 각 언론사에서 나타나는 댓글의 특징과 감성분석</h1>
<h2>
<ul style="list-style-type: none">
    <li style="">202304305 정성현</li>
    <li style="">202104234 안성준</li>
    <li style="">202104241 여동수</li>
</ul>
</h2>
<hr>
<h2 style="margin:5px;" >차례</h2>
<h3>
<ol>
  <li> 들어가며</li>
  <li> 데이터 소개</li>
  <li> 데이터 수집 방법-크롤링</li>
  <li> 데이터 분석-댓글 작성자의 언론사 편향</li>
  <li> 데이터 분석- 댓글 작성 시간</li>
  <li> 데이터 분석- 감정 분석</li>
  <li> 데이터 분석-</li>
  <li> 결론 </li>
  <li> 마치며 </li>
</ol>
<h3>


<h2> 1. 들어가며 </h2>

<p><img src="./notebook_img/EAI_2025_1.png" alt="이미지 오류">
<img src="./notebook_img/EAI_2025_2.png" alt="이미지 오류">
</p>






<h2> 2.데이터 소개 </h2>

성공: 나경원 "한동훈, 도저히 같이 할 수 없는 후보…탄핵 선동" , https://n.news.naver.com/article/015/0005118778?sid=100 , 2025-04-14 , 성공: 1939 실패: 32 총 댓글 개수: 731294


In [19]:
before_integrated=pd.read_csv("../src/news_comments/JTBC/2024-12-09_JTBC_김상욱 _탄핵 찬성..논의 중인 의원들 있어, 탄핵안 통과 충분_.csv")

In [20]:
before_integrated.head(5)

,user_nickname,userIdNo,comments,visible,replyLevel,regTime,parentCommentNo,sympathyCount,antipathyCount,replyAllCount
0,kang****,2uUqJ,김상욱 의원님 화이팅 입니다. 힘내세요. 국민들이 기억할 것 입니다.,True,1,2024-12-10T10:38:22+0900,845152822566584479,1270,38,21
1,gsy7****,NaN,NaN,False,1,2024-12-10T10:38:06+0900,845152805487378484,0,0,16
2,hyun****,4nbAN,다행이다 젊은 보수가 있다는게 \n응원합니다 그래야 보수가 살수있어요\n견제도 할수...,True,1,2024-12-10T10:38:37+0900,845152838974701852,328,10,3
3,syki****,59jX,이래야 국회의원이라할수있지.. 객관적이고 이성적인 판단에 박수를 보냅니다,True,1,2024-12-10T10:39:28+0900,845152893114777828,219,6,3
4,anto****,2mW3x,머털도사와 108요괴라는 만화가 있다 거기서 108요괴가 어떻게 퇴치되는지 보면 저...,True,1,2024-12-10T10:38:48+0900,845152850064441741,105,2,6


In [27]:
df=pd.read_csv("../src/integrated_csv/integrated_csv.csv")

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731294 entries, 0 to 731293
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   press_name       731191 non-null  object
 1   date             731294 non-null  object
 2   user_nickname    731294 non-null  object
 3   userIdNo         575498 non-null  object
 4   comments         575498 non-null  object
 5   visible          731294 non-null  bool  
 6   replyLevel       731294 non-null  int64 
 7   regTime          731294 non-null  object
 8   parentCommentNo  731294 non-null  int64 
 9   sympathyCount    731294 non-null  int64 
 10  antipathyCount   731294 non-null  int64 
 11  replyAllCount    731294 non-null  int64 
dtypes: bool(1), int64(5), object(6)
memory usage: 62.1+ MB


In [42]:
df.query('visible == False').head(1)

,press_name,date,user_nickname,userIdNo,comments,visible,replyLevel,regTime,parentCommentNo,sympathyCount,antipathyCount,replyAllCount
11,중앙일보,2025-01-09,dewy****,NaN,NaN,False,1,2025-01-09T17:37:14+0900,847962946872016996,0,0,0


In [43]:
df.head()

,press_name,date,user_nickname,userIdNo,comments,visible,replyLevel,regTime,parentCommentNo,sympathyCount,antipathyCount,replyAllCount
0,중앙일보,2025-01-09,seki****,6boTo,"여론에는 흔들리지 않겠지만, 전라도 민주당의 지침을 따를 거면서,,,,걸래같은 것들...",True,1,2025-01-09T16:22:50+0900,847958153520742764,154,23,14
1,중앙일보,2025-01-09,lafa****,ddbVw,헌재가 수사 기록을 송부받은 것은 내란죄를 빼고 심리 하겠다는 것입니다. 지금 헌재...,True,1,2025-01-09T16:18:07+0900,847957849668583484,70,14,5
2,중앙일보,2025-01-09,dean****,1CqXM,헌재 탄핵심판은 180일 이내에 결론내면 된다. 왜 이렇게 심리 속도가 느리냐고 따...,True,1,2025-01-09T16:24:48+0900,847958280071283034,55,11,4
3,중앙일보,2025-01-09,alfn****,4GZw0,"이재명 대표의 재판은 너무 이례적이게 늦는거 같고, 민주당이 신청한 국무위원들의 탄...",True,1,2025-01-09T16:27:23+0900,847958446803255471,40,1,0
4,중앙일보,2025-01-09,th******,Drxg,이재명 재판하고 비교해보면 양심이 흔들릴거다...,True,1,2025-01-09T16:24:31+0900,847958261297578225,21,3,1


In [38]:
df['comments'].notnull().sum() + df.query('visible == False').shape[0]

731294

In [30]:
df.describe(
    include="object"
)

,press_name,date,user_nickname,userIdNo,comments,regTime
count,731191,731294,731294,575498,575498,731294
unique,57,132,84115,187550,513611,615907
top,연합뉴스,2024-12-27,park****,3leoB,응원합니다,2024-12-27T11:38:02+0900
freq,185404,19742,2262,147,125,11


<h2> 2. 크롤링 </h2>

In [ ]:

# 사용자 에이전트 정의
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36'
}
crawling_succ_count=0
crawling_fail_count=0
total_crawling_comment=0

# 뉴스 URL 수집 함수
def news_url(query, date1) -> tuple[list, list, datetime.date]:
    page = 1
    url_list = []
    title_list = []
    link_set = set()
    one_day = datetime.timedelta(days=1)
    date2= date1+ one_day
    date1_str= str(date1).replace("-", ".")
    date2_str=str(date2).replace("-", ".")
    date1_str_not_separated= str(date1).replace("-", "")
    date2_str_not_separated= str(date2).replace("-", "")



    while page <= 150:
        #url에 대해
        # url = (
        #         "https://m.search.naver.com/search.naver?where=m_news&sm=tab_pge&query="
        #         + query +
        #         "&sort=0&photo=0&field=0&pd=1&ds=&de=&cluster_rank=129&mynews=0&office_type=0"
        #         "&office_section_code=0&news_office_checked=&nso=so:r,p:1w,a:all&start=" + str(page) #start은 최소 1 최대 1000임
        # )
        url= (
                "https://m.search.naver.com/search.naver?ssc=tab.m_news.all&query="
                + query + "&sm=mtb_opt&sort=0&photo=0&field=0&pd=3&ds="
                + date1_str + "&de=" + date2_str + "&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom"
                + date1_str_not_separated + "to" + date1_str_not_separated + "&is_sug_officeid=0&office_category=0&service_area=0" + str(page)
        )
        headers={
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
            'referer':url
        }

        response = requests.get(url, headers=headers)
        html = response.text
        soup = BeautifulSoup(html, "lxml")
        # atags = soup.select('.news_tit')
        #위에 atags=soup.select('.news_tit')은 위 response로 받아온 html에서 뉴스 링크가 있는 a태그에 있는 클래스였는데 4/28일 확인해보니까
        #이부분이 암호화 되어서 이 코드가 더 이상 유효하지 않는거 같습니다.
        atags = soup.select('._AuHeQ05X7PwSlhb6H2B') #처럼 암호화 된 부분을 확인해서 넣으면 작동하긴 하는데 이게 나중에 어떻게 변할지 모르겠습니다.
        print(atags)
        for i in atags:
            if "https://n.news.naver.com/" in i['href']:
                url_list.append(i['href'])
                news_title_name=get_title_name(i)
                title_list.append(news_title_name)
        if len(atags) < 15:
            break
        # page += 15
        page+= 150
        time.sleep(0.5)

    return url_list ,title_list, date1  #zip으로 묶거나 받는 변수를 2개 나 튜플로 해야함

<img src="./notebook_img/naver_news.png">

In [44]:

#언론사 id(oid_2)를 언론사 이름으로 바꾸는 함수  ex) "023" -> "연합뉴스"
# html요청 없이 따로 딕셔너리 형태로 만들어서 사용하면  더 빨라지는데 언론사 수가 생각보다 많아서 그냥 이렇게 했습니다.
def get_press_name(press_id):
    url_company='https://news.naver.com/main/officeList.naver'
    html_company = urllib.request.urlopen(url_company).read()
    soup_company = BeautifulSoup(html_company, "lxml")
    title_company=soup_company.find_all(class_='list_press nclicks(\'rig.renws2pname\')')
    for i in title_company:
        parts= urlparse(i.attrs['href'])
        if parse_qs(parts.query)['officeId'][0]==press_id:
            news_name = i.text.strip()
            return news_name


# 뉴스 이름을 html에서 가져오는 함수- news_url 함수 안에서 사용해서 별도의 html요청 없이 링크를 가져오려고 하는 html에서 추출
def get_title_name(atag):
    news_title_name=atag.find('span').text.strip()
    print(news_title_name) #가져 오는 뉴스 이름 확인용
    return news_title_name



# 댓글 수집 함수
def comment(url_list,news_title_list, news_date):
    global crawling_succ_count
    global crawling_fail_count
    global total_crawling_comment
    url_total_num= len(url_list)
    url_list_num=0
    news_title_list_num=0
    total_comment = [];user_nickname=[];comments=[];userIdNo=[];visible=[];replyLevel=[];regTime=[];parentCommentNo=[];sympathyCount=[];antipathyCount=[];replyAllCount=[]
    # following=[]
    for url_ex in url_list:
        url = url_ex.split('?')[0]
        oid_1 = url.split('/')[-1]
        oid_2 = url.split('/')[-2]
        print(news_title_list[news_title_list_num],url_list[url_list_num])
        i = 1

        while True:
            params = {
                'ticket': 'news',
                'templateId': 'default_society',
                'pool': 'cbox5',
                'lang': 'ko',
                'country': 'KR',
                'objectId': f'news{oid_2},{oid_1}',
                'pageSize': '100',
                'indexSize': '10',
                'page': str(i),
                'currentPage': '0',
                'moreParam.direction': 'next',
                'moreParam.prev': '',
                'moreParam.next': '',
                'followSize': '100',
                'includeAllStatus': 'true',
            }
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
                'referer': url_ex
            }
            response = requests.get(
                'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json',
                params=params,
                headers=headers
            )

            response.encoding = "UTF-8-sig"
            res = response.text.replace("_callback(", "")[:-2]


            try:
                temp = json.loads(res)
                comment_list = temp['result'].get('commentList', [])
                total_crawling_comment+=len(comment_list)
                if not comment_list:
                    print("comment 가져오기 실패")
                    crawling_fail_count+=1
                    log_text(
                        news_title_list[news_title_list_num],
                        url_list[url_list_num],
                        news_date,crawling_succ_count,
                        crawling_fail_count,
                        total_crawling_comment,
                        2)
                    url_list_num += 1
                    news_title_list_num += 1
                    break
                #아래는 가져온 comment_list( 위에서 요청한 json 내용 으로 'contents': '탄해반대집회는~~' 형태로 존재)

                # if ((not user_nickname) and (not comments)):
                #     user_nickname = [k['userName'] for k in comment_list]  # json 내용에서 userName에 해당하는 부분을 가져와 리스트로 저장시킴
                #     comments = [c['contents'] for c in comment_list]  # json에서 contents에 해당하는 부분을 가져와 리스트로 저장시킴
                #     # hiddenByCleanbot= [a['hiddenByCleanbot'] for a in comment_list]
                #     userIdNo=[b['userIdNo'] for b in comment_list]
                #     visible= [c1['visible'] for c1 in comment_list]
                #     replyLevel=[d['replyLevel'] for d in comment_list]
                #     regTime=[e['regTime'] for e in comment_list]
                #     parentCommentNo=[f['parentCommentNo'] for f in comment_list]
                #     sympathyCount= [g['sympathyCount'] for g in comment_list]
                #     antipathyCount=[h['antipathyCount'] for h in comment_list]
                #     # following=[p['following'] for p in comment_list]
                #     replyAllCount=[r['replyAllCount'] for r in comment_list]
                #
                #
                #
                # else:
                #     user_nickname.extend([k['userName'] for k in comment_list])
                #     comments.extend([c['contents'] for c in comment_list])
                #     # hiddenByCleanbot.extend= [a['hiddenByCleanbot'] for a in comment_list]
                #     userIdNo.extend([b['userIdNo'] for b in comment_list])
                #     visible.extend([c['visible'] for c in comment_list])
                #     replyLevel.extend([d['replyLevel'] for d in comment_list])
                #     regTime.extend([e['regTime'] for e in comment_list])
                #     parentCommentNo.extend([f['parentCommentNo'] for f in comment_list])
                #     sympathyCount.extend ([g['sympathyCount'] for g in comment_list])
                #     antipathyCount.extend([h['antipathyCount'] for h in comment_list])
                #     # following.extend([p['following'] for p in comment_list])
                #     replyAllCount.extend(r['replyAllCount'] for r in comment_list)
                if not user_nickname and not comments:
                    user_nickname = []
                    comments = []
                    userIdNo = []
                    visible = []
                    replyLevel = []
                    regTime = []
                    parentCommentNo = []
                    sympathyCount = []
                    antipathyCount = []
                    replyAllCount = []
                for item in comment_list:
                    user_nickname.append(item['userName'])
                    comments.append(item['contents'])
                    userIdNo.append(item['userIdNo'])
                    visible.append(item['visible'])
                    replyLevel.append(item['replyLevel'])
                    regTime.append(item['regTime'])
                    parentCommentNo.append(item['parentCommentNo'])
                    sympathyCount.append(item['sympathyCount'])
                    antipathyCount.append(item['antipathyCount'])
                    replyAllCount.append(item['replyAllCount'])


                if len(comment_list) < 100:
                    #total_comment.extend(comments) #없어도 되는 코드 gptexample에 있던 코드
                    comment_list_sum=list(
                        zip(
                            user_nickname,
                            userIdNo,
                            comments,
                            visible,
                            replyLevel,
                            regTime,
                            parentCommentNo,
                            sympathyCount,
                            antipathyCount,
                            replyAllCount
                        )
                    ) #df로 저장하기 위해 묶어줌
                    col=['user_nickname','userIdNo','comments','visible','replyLevel','regTime','parentCommentNo','sympathyCount','antipathyCount','replyAllCount']
                    df=pd.DataFrame(comment_list_sum, columns=col) # columns를 작성자, 내용 으로 가지는 df생성
                    #파일을 저장하기 위한 경로 이름 생성, 파일 이름 생성
                    news_name=get_press_name(oid_2) #언론사 id를 언론사 이름 str로 바꾸는 함수
                    BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
                    folder_path = os.path.join(BASE_DIR, "src", "news_comments",news_name)
                    # folder_path= f'../src/news_comments/{news_name}'
                    # folder_path=f'./{news_name}'
                    os.makedirs(folder_path, exist_ok=True) #이 py가 있는 경로에 "언론사이름"을 이름으로 하는 폴더 생성, 존재시 넘어감
                    # print(news_title_list_num)

                    if news_title_list_num >= len(news_title_list):
                        print(f"news_title_list_num{news_title_list_num}이 news_title_list {len(news_title_list)}길이를 초과했습니다.")
                        url_list_num += 1
                        news_title_list_num += 1
                        # url_list_num = 0
                        # news_title_list_num = 0
                        break
                    newstitle_regex= re.sub(r'[\\/*?:"<>|]', '_',news_title_list[news_title_list_num])
                    file_name= f"{news_date}_{news_name}_{newstitle_regex}.csv"
                    folder_file_path= os.path.join(folder_path, file_name)
                    #파일 이름을 언론사_뉴스제목 으로 정의
                    # if os.path.exists(f"{folder_path}/{file_name}"): #같은 이름 존재시 break
                    #     print(f"{folder_path}/{file_name} 이미 존재함. ")

                    if os.path.exists(folder_file_path):
                        print(f"{folder_file_path} 이미 존재함.")
                        log_text(
                            news_title_list[news_title_list_num],
                            url_list[url_list_num],
                            news_date,
                            crawling_succ_count,
                            crawling_fail_count,
                            total_crawling_comment,
                            3
                        )
                        url_list_num += 1
                        news_title_list_num += 1
                        continue
                    df.to_csv(f"{folder_path}/{file_name}", index=False)
                    comment_list_sum = []
                    comments=[]
                    user_nickname= []
                    # hiddenByCleanbot= []
                    userIdNo=[];visible= []
                    replyLevel=[];regTime=[];replyAllCount=[]
                    parentCommentNo=[];sympathyCount=[];antipathyCount=[];following=[]

                    print(f"\n{news_date}: {news_title_list_num+1}/{url_total_num}")
                    crawling_succ_count+=1
                    print(f'성공: {crawling_succ_count} 실패: {crawling_fail_count}')
                    log_text(
                        news_title_list[news_title_list_num],
                        url_list[url_list_num],
                        news_date,
                        crawling_succ_count,
                        crawling_fail_count,
                        total_crawling_comment,
                        1
                        )
                    url_list_num += 1
                    news_title_list_num += 1
                    break
                else:
                    i += 1 # params= ..., 'page'=str(i),...에서  pageSize=100은 한 페이지당 불러올 댓글 최대가 100이기 떄문에 다음페이지로
                    #넘겨야함 즉 i 값 증가시켜서 다음 페이지 댓글 최대 100개를 불러옴
                    time.sleep(0.3)
            except Exception as e:
                print(f"[에러 발생] {url_ex}: {e}")
                break
            finally:
                print('|',end="")

    return total_comment #필요 없음
def log_text(news_url,news_title,news_date,succ_count,fail_count,total_crawling_comment,current_status):
    base_dir= os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
    target_path= os.path.join(base_dir, "src", "news_comments", "into.txt")
    if (current_status==1):
        with open(target_path, 'a', encoding='utf-8') as file:
            file.write(f'성공: {news_url} , {news_title} , {news_date} , 성공: {succ_count} 실패: {fail_count} 총 댓글 개수: {total_crawling_comment}\n')
    elif  (current_status==2):
        with open(target_path, 'a', encoding='utf-8') as file:
            file.write( f'실패: {news_url} , {news_title} , {news_date} , 성공: {succ_count} 실패: {fail_count} 총 댓글 개수: {total_crawling_comment}\n')
    elif(current_status==3):
        with open(target_path, 'a', encoding='utf-8') as file:
            file.write( f'중복: {news_url} , {news_title} , {news_date} , 성공: {succ_count} 실패: {fail_count} 총 댓글 개수: {total_crawling_comment}\n')

query = "윤석열 탄핵"  # 원하는 검색어
encoded_query = urllib.parse.quote(query) #url에 쿼리스트링에 한글 사용하려면 인코딩 필수
date= datetime.date(2024, 12, 4)
while(date< datetime.date(2025, 4, 15)):
    news_links, news_title ,news_date= news_url(encoded_query, date)
    print(f"{date} 뉴스 url 수집 완료")
    comment(news_links, news_title, news_date)
    print(news_links)
    date= date+ datetime.timedelta(days=1)

992

<img src="./notebook_img/explain_code.jpeg" width=720 height=360>

In [ ]:
target_path= "../src/news_comments"
folders=[
    name for name in os.listdir(target_path)
    if os.path.isdir(os.path.join(target_path))
]
print(folders)
df_all=pd.DataFrame()
count=0
for folder in folders:
    path=f"../{target_path}/{folder}"
    try:
        in_folder= [
        name for name in os.listdir(path)
        if os.path.isdir(os.path.join(path))
        ]
        print(in_folder)
        print(f"{folder}: {len(in_folder)}")
    except NotADirectoryError as e:
        continue
    for csv_name in in_folder:
        count+=1
        try:
            date=re.findall(r"\d{4}-\d{2}-\d{2}",csv_name)[0]
            df_read = pd.read_csv(f"{path}/{csv_name}")
            df_read.insert(0, "date", date)
            df_read.insert(0, "press_name", folder)
            df_all = pd.concat([df_all, df_read])
            print(f"{count} \n {folder}-{csv_name} 통합 완료 ")
            with open("./log_integrating_csv4_all.txt", "a") as file:
                file.write(f"{count}{in_folder}-{csv_name} 통합 완료 \n")
        except IndexError as e:
            print(e)
df_all.to_csv(f"../src/integrated_csv/integrated_csv.csv",index=False)

<h2> 데이터 분석-댓글 작성자의 언론사 편향</h2>
